# High-Frequency Index Arbitrage

## Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.tsa.vector_ar.vecm import coint_johansen

## Load Data

In [2]:
DJIAdf = pd.read_csv('data/DJIA_history.csv', skiprows=1)
DJIAdf.columns = ['Date', 'Close', 'Volume', 'Open', 'High', 'Low']
DJIAdf['Date'] = pd.to_datetime(DJIAdf['Date'], errors='coerce')
DJIAdf['Close'] = pd.to_numeric(DJIAdf['Close'], errors='coerce')

DOWdf = pd.read_csv('data/DOW_daily_history.csv', header=0)
DOWdf['Date'] = pd.to_datetime(DOWdf['Date'], errors='coerce')
DOWdf.set_index('Date', inplace=True)

stocks = ['DIA','GS.N', 'NKE.N', 'CSCO.OQ', 'JPM.N', 'DIS.N', 'INTC.OQ', 'DOW.N', 'MRK.N', 'CVX.N', 'AXP.N', 'VZ.N', 'HD.N', 'WBA.OQ', 'MCD.N', 'UNH.N', 'KO.N', 'JNJ.N', 'MSFT.OQ', 'HON.OQ', 'CRM.N', 'PG.N', 'IBM.N', 'MMM.N', 'AAPL.OQ', 'WMT.N', 'CAT.N', 'AMGN.OQ', 'V.N', 'TRV.N', 'BA.N']

## Cointegrating stocks

In [3]:
def find_pairs(data, significance=0.1):
    n = data.shape[1]
    score_matrix = np.zeros((n, n))
    pvalue_matrix = np.ones((n, n))
    keys = data.keys()
    pairs = []
    for i in range(n):
        for j in range(i+1, n):
            stock1 = data[keys[i]]
            stock2 = data[keys[j]]
            result = coint_johansen(data[[keys[i], keys[j]]].dropna(), det_order=0, k_ar_diff=1)
            score = result.lr1[0]
            pvalue = result.cvt[0, 1] 
            score_matrix[i, j] = score
            pvalue_matrix[i, j] = pvalue
            if pvalue < significance:
                pairs.append((keys[i], keys[j]))
    return pairs

training_data = DOWdf[DOWdf.index.year == 2021]
cointegrating_pairs = find_pairs(training_data)

## Constructing portfolio

In [4]:
portfolio_stocks = [pair[0] for pair in cointegrating_pairs]
portfolio_data = training_data[portfolio_stocks]
portfolio_returns = portfolio_data.pct_change().dropna()

# Check cointegration

DJIAdf.set_index('Date', inplace=True)
DJIAdf_pct_change = DJIAdf['Close'].pct_change()
DJIAdf_pct_change.sort_index(inplace=True)
DJIAdf_pct_change_filtered = DJIAdf_pct_change['2021-01-04':'2021-12-31']

print(portfolio_returns.head(10))
print(portfolio_data.head(10))
print(portfolio_data.isnull().sum())

combined_data = pd.concat([DJIAdf_pct_change_filtered, portfolio_returns.mean(axis=1)], axis=1, join='inner').dropna()

result = coint_johansen(combined_data, det_order=0, k_ar_diff=1)
is_cointegrated = result.cvt[0, 1] > 0.1

Empty DataFrame
Columns: []
Index: [2021-01-04 00:00:00, 2021-01-05 00:00:00, 2021-01-06 00:00:00, 2021-01-07 00:00:00, 2021-01-08 00:00:00, 2021-01-11 00:00:00, 2021-01-12 00:00:00, 2021-01-13 00:00:00, 2021-01-14 00:00:00, 2021-01-15 00:00:00]
Empty DataFrame
Columns: []
Index: [2021-01-04 00:00:00, 2021-01-05 00:00:00, 2021-01-06 00:00:00, 2021-01-07 00:00:00, 2021-01-08 00:00:00, 2021-01-11 00:00:00, 2021-01-12 00:00:00, 2021-01-13 00:00:00, 2021-01-14 00:00:00, 2021-01-15 00:00:00]
Series([], dtype: float64)


ValueError: zero-size array to reduction operation maximum which has no identity

## Mean-reversion strategy

In [ ]:
def mean_reversion_strategy(data):
    data_cleaned = data.copy().dropna()
    delta_y = data_cleaned.diff().dropna()
    lagged_data = data_cleaned.shift(1).dropna()

    delta_y = delta_y['Close']
    lagged_data = lagged_data['Close']

    exog = sm.add_constant(lagged_data)
    model = sm.OLS(delta_y, exog)
    results = model.fit()

    lambda_val = results.params[1]
    half_life = -np.log(2) / lambda_val

    data['MovingAvg'] = data['Close'].rolling(window=int(half_life)).mean()
    data['MovingStd'] = data['Close'].rolling(window=int(half_life)).std()
    data['Z'] = (data['Close'] - data['MovingAvg']) / data['MovingStd']
    data['Position'] = -data['Z']

    return data

strategy_data = mean_reversion_strategy(DJIAdf)

## Backtest